In [ ]:
pip install tensorflow numpy

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the images to [0, 1] range
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_cnn(architecture):
    model = Sequential()

    # First layer is mandatory for the input layer
    first_layer = architecture[0]
    if first_layer['type'] == 'conv':
        model.add(Conv2D(first_layer['filters'], kernel_size=(first_layer['kernel_size'], first_layer['kernel_size']),
                         activation='relu', input_shape=(32, 32, 3)))

    # Add remaining layers based on the architecture specification
    for layer in architecture[1:]:
        if layer['type'] == 'conv':
            model.add(Conv2D(layer['filters'], kernel_size=(layer['kernel_size'], layer['kernel_size']), activation='relu'))
        elif layer['type'] == 'maxpool':
            model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))  # Output layer for 10 classes

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def fitness_function(architecture):
    try:
        model = build_cnn(architecture)
        model.fit(x_train, y_train, epochs=1, batch_size=64, verbose=0)
        _, accuracy = model.evaluate(x_test, y_test, verbose=0)
        return accuracy
    except Exception as e:
        print(f"Error during fitness evaluation: {e}")
        return 0  # Return zero if an error occurs


In [ ]:
import random

In [ ]:
def create_population(pop_size):
    population = []
    for _ in range(pop_size):
        architecture = []
        num_layers = random.randint(2, 4)  # Random number of layers (2 to 4 layers)

        for _ in range(num_layers):
            layer_type = random.choice(['conv', 'maxpool'])
            if layer_type == 'conv':
                architecture.append({'type': 'conv', 'filters': random.choice([32, 64, 128]),
                                     'kernel_size': random.choice([3, 5, 7])})
            elif layer_type == 'maxpool':
                architecture.append({'type': 'maxpool'})
        population.append(architecture)
    return population

In [ ]:
def tournament_selection(population, fitness_values, tournament_size=3):
    selected = random.sample(list(zip(population, fitness_values)), tournament_size)
    selected.sort(key=lambda x: x[1], reverse=True)  # Sort by fitness (accuracy)
    return selected[0][0]  # Return the best individual


In [ ]:
def crossover(parent1, parent2):
    crossover_point = random.randint(0, min(len(parent1), len(parent2)) - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

In [ ]:
def mutate(architecture):
    if random.random() < 0.1:  # 10% chance of mutation
        mutate_index = random.randint(0, len(architecture) - 1)
        if architecture[mutate_index]['type'] == 'conv':
            architecture[mutate_index]['filters'] = random.choice([32, 64, 128])
            architecture[mutate_index]['kernel_size'] = random.choice([3, 5, 7])
        else:
            # Add a new conv layer (mutation)
            architecture.append({'type': 'conv', 'filters': random.choice([32, 64, 128]), 'kernel_size': random.choice([3, 5, 7])})
    return architecture


In [ ]:
def genetic_algorithm(pop_size=10, generations=5):
    # Initialize population
    population = create_population(pop_size)

    for generation in range(generations):
        print(f"Generation {generation + 1}:")

        # Evaluate fitness for each architecture
        fitness_values = [fitness_function(arch) for arch in population]
        print(f"Fitness values: {fitness_values}")

        # Select the best architectures based on fitness
        selected_individuals = [tournament_selection(population, fitness_values) for _ in range(pop_size)]

        # Create the next generation via crossover and mutation
        next_generation = []
        while len(next_generation) < pop_size:
            parent1, parent2 = random.sample(selected_individuals, 2)
            child = crossover(parent1, parent2)
            child = mutate(child)
            next_generation.append(child)

        population = next_generation

    # Return the best architecture found
    fitness_values = [fitness_function(arch) for arch in population]
    best_architecture = population[fitness_values.index(max(fitness_values))]
    return best_architecture


In [ ]:
best_architecture = genetic_algorithm(pop_size=10, generations=5)
print(f"Best Architecture Found: {best_architecture}")

Generation 1:


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fitness values: [0.32190001010894775, 0.4894999861717224, 0.42640000581741333, 0.5526000261306763, 0.47450000047683716, 0.43970000743865967, 0.5056999921798706, 0.48579999804496765, 0.3215999901294708, 0.5421000123023987]
Generation 2:
Fitness values: [0.5685999989509583, 0.5598000288009644, 0.49869999289512634, 0.5128999948501587, 0.5016999840736389, 0.5386999845504761, 0.5041999816894531, 0.5358999967575073, 0.46709999442100525, 0.5095000267028809]
Generation 3:
Fitness values: [0.5206000208854675, 0.4691999852657318, 0.5436000227928162, 0.5590999722480774, 0.474700003862381, 0.5572999715805054, 0.5480999946594238, 0.5817000269889832, 0.5141000151634216, 0.5232999920845032]
Generation 4:
Fitness values: [0.5412999987602234, 0.5259000062942505, 0.49380001425743103, 0.5644999742507935, 0.5077000260353088, 0.5486000180244446, 0.5626000165939331, 0.4875999987125397, 0.5428000092506409, 0.5340999960899353]
Generation 5:
Fitness values: [0.5411999821662903, 0.5439000129699707, 0.5464000105

In [ ]:
# Test Case 1: Verify Architecture Creation
def test_build_cnn():
    architecture = [{'type': 'conv', 'filters': 32, 'kernel_size': 3},
                    {'type': 'conv', 'filters': 64, 'kernel_size': 3},
                    {'type': 'maxpool'}]

    model = build_cnn(architecture)
    assert isinstance(model, tf.keras.models.Sequential), "Model creation failed!"
    print("test_build_cnn passed!")

In [ ]:
# Test Case 2: Verify Fitness Function
def test_fitness_function():
    architecture = [{'type': 'conv', 'filters': 32, 'kernel_size': 3},
                    {'type': 'maxpool'}]
    accuracy = fitness_function(architecture)
    assert accuracy >= 0, f"Invalid fitness value: {accuracy}"
    print("test_fitness_function passed!")

In [ ]:
# Test Case 3: Verify Population Initialization
def test_create_population():
    population = create_population(5)
    assert len(population) == 5, f"Population size mismatch: {len(population)}"
    assert all(isinstance(individual, list) for individual in population), "Population contains non-list architectures"
    print("test_create_population passed!")

In [ ]:
# Test Case 4: Verify GA Process Runs
def test_genetic_algorithm():
    best_architecture = genetic_algorithm(pop_size=5, generations=2)
    assert isinstance(best_architecture, list), "Best architecture is not a list"
    print("test_genetic_algorithm passed!")

In [ ]:
# Test Case 5: Verify Model Evaluation
def test_model_evaluation():
    architecture = [{'type': 'conv', 'filters': 32, 'kernel_size': 3},
                    {'type': 'maxpool'}]

    model = build_cnn(architecture)
    model.fit(x_train, y_train, epochs=1, batch_size=64, verbose=0)  # Train for one epoch
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)

    assert accuracy > 0.1, f"Model evaluation failed! Accuracy: {accuracy}"
    print("test_model_evaluation passed!")

In [ ]:
# Running all test cases
test_build_cnn()
test_fitness_function()
test_create_population()
test_genetic_algorithm()
test_model_evaluation()

test_build_cnn passed!
test_fitness_function passed!
test_create_population passed!
Generation 1:
Fitness values: [0.5715000033378601, 0.45089998841285706, 0.5406000018119812, 0.6162999868392944, 0.4745999872684479]
Generation 2:
Fitness values: [0.6000000238418579, 0.5956000089645386, 0.5978000164031982, 0.5257999897003174, 0.5360999703407288]
test_genetic_algorithm passed!
test_model_evaluation passed!
